This notebook goes through an EDA of a dataset of adult income using Postgres. The link to the Kaggle dataset is https://www.kaggle.com/datasets/armanmanteghi/data-analytics-with-python-with-demographics, which itself is taken from the UCI Adult Income dataset (located at https://archive.ics.uci.edu/dataset/2/adult).

As is typical, imports first:

In [1]:
import pandas as pd

# for loading dataset as pandas df into SQL database
from sqlalchemy import create_engine

# Initializing PostgreSQL and Loading Data

Create the database to start (and deleting it first if it already exists by running this notebook from scratch).

In [ ]:
%%bash
#pg_ctl restart -D /usr/local/var/postgres
psql -c "DROP DATABASE IF EXISTS ADULT_DATA;" postgres
psql -c "CREATE DATABASE ADULT_DATA;" postgres

Load postgres and the initialized database

In [3]:
%load_ext sql
%sql postgresql:///adult_data

Connecting to 'postgresql:///adult_data'

Run a test command to check that postgres has been loaded correctly.

In [4]:
%sql SELECT current_date;

Running query in 'postgresql:///adult_data'

1 rows affected.

current_date
2024-10-11


Perfect! Now let's load the dataset into a database using pandas and sqalchemy.

In [5]:
df = pd.read_csv('adult_data.csv')
engine = create_engine('postgresql://localhost/adult_data', echo=False)
df.to_sql('adult_data', engine, if_exists='replace', index=False)

561

Checking that all rows have been loaded (should be around 32k).

In [6]:
%%sql
SELECT
    COUNT(*)
FROM
    adult_data;

Running query in 'postgresql:///adult_data'

1 rows affected.

count
32561


And setting display limit to None (so that there's no implicit automatic limits of 10 unless we set it explicitly, or anything like that)

In [7]:
%config SqlMagic.displaylimit = None

displaylimit: Value None will be treated as 0 (no limit)

Now we can proceed with the analysis!

# Analysis

Let's first check the available columns and their data types as well as the first few rows of the table.

In [8]:
%%sql
SELECT
    COLUMN_NAME, DATA_TYPE
FROM
    INFORMATION_SCHEMA.COLUMNS
WHERE
    TABLE_NAME = 'adult_data'
;

Running query in 'postgresql:///adult_data'

15 rows affected.

column_name,data_type
fnlwgt,bigint
capital-gain,bigint
capital-loss,bigint
hours-per-week,bigint
education-num,bigint
age,bigint
race,text
sex,text
native-country,text
salary,text


In [10]:
%%sql
SELECT
    *
FROM
    adult_data
LIMIT
    10;

Running query in 'postgresql:///adult_data'

10 rows affected.

age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
37,Private,284582,Masters,14,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,United-States,<=50K
49,Private,160187,9th,5,Married-spouse-absent,Other-service,Not-in-family,Black,Female,0,0,16,Jamaica,<=50K
52,Self-emp-not-inc,209642,HS-grad,9,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,45,United-States,>50K
31,Private,45781,Masters,14,Never-married,Prof-specialty,Not-in-family,White,Female,14084,0,50,United-States,>50K
42,Private,159449,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,5178,0,40,United-States,>50K


Looks like there's variety of categories across most of the categorical data. Let's define a function that gets the unique categories of a specified column, their counts (as raw ints and percentages), and cumulative percentage of the data.

In [55]:
%%sql

--%sql DROP FUNCTION categ_counts(TEXT); --used when needed to modify output for func!

CREATE OR REPLACE FUNCTION categ_counts(col TEXT)

RETURNS TABLE (
    categ TEXT,
    count BIGINT,
    perc NUMERIC,
    cumul_perc NUMERIC
)

LANGUAGE plpgsql
AS $$
BEGIN
    RETURN QUERY EXECUTE format($q$
                                
        WITH counts AS (
            SELECT
                %I,
                COUNT(*) AS count
            FROM
                adult_data
            GROUP BY
                %I
        ), 
        percs AS (
            SELECT
                %I,
                count,
                ROUND(count * 100.0 / SUM(count) OVER (), 2) AS perc
            FROM
                counts
        )
        SELECT
            %I,
            count,
            perc,
            SUM(perc) OVER (ORDER BY count DESC) AS cumul_perc
        FROM
            percs
        ORDER BY
            count DESC;
                                
    $q$, col, col, col, col);
END;
$$;

Running query in 'postgresql:///adult_data'

++
||
++
++

Here are the distributions of unique categories for each column, sorted by occupancy.

In [56]:
%sql SELECT * FROM categ_counts('workclass') LIMIT 20;

Running query in 'postgresql:///adult_data'

9 rows affected.

categ,count,perc,cumul_perc
Private,22696,69.70,69.70
Self-emp-not-inc,2541,7.80,77.50
Local-gov,2093,6.43,83.93
?,1836,5.64,89.57
State-gov,1298,3.99,93.56
Self-emp-inc,1116,3.43,96.99
Federal-gov,960,2.95,99.94
Without-pay,14,0.04,99.98
Never-worked,7,0.02,100.00


In [57]:
%sql SELECT * FROM categ_counts('education') LIMIT 20;

Running query in 'postgresql:///adult_data'

16 rows affected.

categ,count,perc,cumul_perc
HS-grad,10501,32.25,32.25
Some-college,7291,22.39,54.64
Bachelors,5355,16.45,71.09
Masters,1723,5.29,76.38
Assoc-voc,1382,4.24,80.62
11th,1175,3.61,84.23
Assoc-acdm,1067,3.28,87.51
10th,933,2.87,90.38
7th-8th,646,1.98,92.36
Prof-school,576,1.77,94.13


In [58]:
%sql SELECT * FROM categ_counts('marital-status') LIMIT 20;

Running query in 'postgresql:///adult_data'

7 rows affected.

categ,count,perc,cumul_perc
Married-civ-spouse,14976,45.99,45.99
Never-married,10683,32.81,78.80
Divorced,4443,13.65,92.45
Separated,1025,3.15,95.60
Widowed,993,3.05,98.65
Married-spouse-absent,418,1.28,99.93
Married-AF-spouse,23,0.07,100.00


In [59]:
%sql SELECT * FROM categ_counts('occupation') LIMIT 20;

Running query in 'postgresql:///adult_data'

15 rows affected.

categ,count,perc,cumul_perc
Prof-specialty,4140,12.71,12.71
Craft-repair,4099,12.59,25.30
Exec-managerial,4066,12.49,37.79
Adm-clerical,3770,11.58,49.37
Sales,3650,11.21,60.58
Other-service,3295,10.12,70.70
Machine-op-inspct,2002,6.15,76.85
?,1843,5.66,82.51
Transport-moving,1597,4.90,87.41
Handlers-cleaners,1370,4.21,91.62


In [60]:
%sql SELECT * FROM categ_counts('relationship') LIMIT 20;

Running query in 'postgresql:///adult_data'

6 rows affected.

categ,count,perc,cumul_perc
Husband,13193,40.52,40.52
Not-in-family,8305,25.51,66.03
Own-child,5068,15.56,81.59
Unmarried,3446,10.58,92.17
Wife,1568,4.82,96.99
Other-relative,981,3.01,100.00


In [61]:
%sql SELECT * FROM categ_counts('race') LIMIT 20;

Running query in 'postgresql:///adult_data'

5 rows affected.

categ,count,perc,cumul_perc
White,27816,85.43,85.43
Black,3124,9.59,95.02
Asian-Pac-Islander,1039,3.19,98.21
Amer-Indian-Eskimo,311,0.96,99.17
Other,271,0.83,100.00


In [62]:
%sql SELECT * FROM categ_counts('sex') LIMIT 20;

Running query in 'postgresql:///adult_data'

2 rows affected.

categ,count,perc,cumul_perc
Male,21790,66.92,66.92
Female,10771,33.08,100.00


In [63]:
%sql SELECT * FROM categ_counts('native-country') LIMIT 20;

Running query in 'postgresql:///adult_data'

20 rows affected.

categ,count,perc,cumul_perc
United-States,29170,89.59,89.59
Mexico,643,1.97,91.56
?,583,1.79,93.35
Philippines,198,0.61,93.96
Germany,137,0.42,94.38
Canada,121,0.37,94.75
Puerto-Rico,114,0.35,95.10
El-Salvador,106,0.33,95.43
India,100,0.31,95.74
Cuba,95,0.29,96.03


In [64]:
%sql SELECT * FROM categ_counts('salary') LIMIT 20;

Running query in 'postgresql:///adult_data'

2 rows affected.

categ,count,perc,cumul_perc
<=50K,24720,75.92,75.92
>50K,7841,24.08,100.00


Across the categorical data, the largest groups are ...

1. privately employed (70%, workclass),

2. have completed high school, completed some college, or got a bachelor's (cumulative 71%, education),

3. currently married or never married (cumulative 79%, marital-status), 

4. have a job described by Prof-specialty, Craft-repair, Exec-managerial, Adm-clerical, Sales, or Other-service (cumulative 77%, occupation),

5. are a Husband, Not-in-family, or their Own-child (cumulative 82%, relationship),

6. are overwhelmingly white (85%, race),

7. are largely male (67%, sex),

8. are predominantly born in the united states (90%, native-country),

9. and earn less than \$50k a year (76%, salary).

Now we'll investigate the numerical data by looking the at the min, max, mean, 25th, 50th \[median\], and 75th percentiles, and the standard deviation per numerical column. Since the structure is consistent across numerical data, we can just define a single function with a for loop to calculate the stats per column and join them all into a single table.

In [101]:
%%sql

--DROP FUNCTION numer_stats(TEXT[]); -- uncomment to delete func

CREATE OR REPLACE FUNCTION numer_stats(cols TEXT[])

RETURNS TABLE (
    col_name TEXT,
    min_ NUMERIC,
    max_ NUMERIC,
    mean NUMERIC,
    perc25 NUMERIC,
    median NUMERIC,
    perc75 NUMERIC,
    std_dev NUMERIC
)

LANGUAGE plpgsql
AS $$
DECLARE
    col TEXT;
BEGIN

    FOREACH col IN ARRAY cols
    LOOP
        RETURN QUERY EXECUTE format($q$
                                    
            SELECT 
                %L AS col_name,
                MIN(%I)::NUMERIC,
                MAX(%I)::NUMERIC,
                ROUND(AVG(%I), 2) AS mean,
                PERCENTILE_CONT(0.25) WITHIN GROUP(ORDER BY %I)::NUMERIC AS perc25,
                PERCENTILE_CONT(0.5) WITHIN GROUP(ORDER BY %I)::NUMERIC AS median,
                PERCENTILE_CONT(0.75) WITHIN GROUP(ORDER BY %I)::NUMERIC AS perc75,
                ROUND(STDDEV_SAMP(%I), 2) AS std_dev
            FROM 
                adult_data;
                                    
        $q$, col, col, col, col, col, col, col, col);
    END LOOP;
    
END;
$$;

Running query in 'postgresql:///adult_data'

++
||
++
++

In [102]:
%%sql
SELECT * 
FROM numer_stats(ARRAY['age','hours-per-week','education-num','fnlwgt','capital-gain','capital-loss']);

Running query in 'postgresql:///adult_data'

6 rows affected.

col_name,min_,max_,mean,perc25,median,perc75,std_dev
age,17,90,38.58,28,37,48,13.64
hours-per-week,1,99,40.44,40,40,45,12.35
education-num,1,16,10.08,9,10,12,2.57
fnlwgt,12285,1484705,189778.37,117827,178356,237051,105549.98
capital-gain,0,99999,1077.65,0,0,0,7385.29
capital-loss,0,4356,87.30,0,0,0,402.96


And the observations for the numerical data is as follows:

1. The age distribution is largely represented by late 20's to late 40's (IQR)

2. The hours-per-week is pretty tight with respect to the percentiles (40 hours is both the 25 and 50 percentiles and 45 is 75 percentile) but has some significant outliers judging from the standard deviation.

3. The education-num is also pretty tight with similar values in the 25, 50, and 75 percentiles and a low standard deviation.

4. fnlwgt (a column that is not explained in either the Kaggle or UCI repository) is very skewed with a huge difference between mean and median.

5. At least 75% of the data for both capital-gain and capital-loss are exactly 0 (seen from the min_ and percentile columns). From the max_, mean, and std_dev, there are a number of large non-zero values.

Now we can proceed by positing some questions

# Questions

1. **What's the relationship between hours-per-week worked and education level?**

2. **Is there a difference in education level between U.S. born and non-U.S. born persons?** 

3. **Does sex affect the likelihood of earning greater than \$50k?**

4. **For those earning a salary >\$50k, is there a change in hours-per-week worked between different racial categories?**

We'll see if any kinds of relationships are suggested here, but we will statistically validate them using R
Note that the answers to these questions *are not* statistically validated here - but will be done in a separate R markdown document (put document title here).

### Q1: What's the relationship between hours-per-week worked and education level?

In [120]:
%%sql
SELECT
    "education-num",
    education,
    COUNT(*) as num,
    ROUND(AVG("hours-per-week"),2) as avg_hours_per_week, --quotes needed b/c hyphens
    ROUND(STDDEV_SAMP("hours-per-week"),2) as std_hours_per_week --quotes needed b/c hyphens
FROM
    adult_data
GROUP BY
    education,
    "education-num" --easy since 1-to-1 correspondence b/w edu-num and edu label
ORDER BY
    avg_hours_per_week DESC;

Running query in 'postgresql:///adult_data'

16 rows affected.

education-num,education,num,avg_hours_per_week,std_hours_per_week
15,Prof-school,576,47.43,14.81
16,Doctorate,413,46.97,15.08
14,Masters,1723,43.84,12.28
13,Bachelors,5355,42.61,11.45
11,Assoc-voc,1382,41.61,10.79
9,HS-grad,10501,40.58,11.33
12,Assoc-acdm,1067,40.50,12.20
4,7th-8th,646,39.37,14.20
3,5th-6th,333,38.90,10.55
10,Some-college,7291,38.85,12.76


The Prof-school education level has the highest average number of hours worked per week at 47.5 hours. Doctorate level is comparable at 47 hours. The average amount begins to drop off after that with 3rd place being Masters level at 44 hours followed by lower education levels having fewer average hours worked per week. It's worth noting that both Prof-school and Doctorate have the 2 largest standard deviations among the groups, so there could be outliers pushing these higher. Various levels of high school education occupy the middle and bottom slots between the 16 groups.

**Q1 Answer: Higher educations seems to be related to a higher average of hours worked per week.**

### Q2: Is there a difference in education level between U.S. born and non-U.S. born persons?

In [166]:
%%sql
-- first get table of non-us born stats
WITH non_us_born as (
    SELECT
        "education-num",
        education,
        COUNT(*) as count
    FROM
        adult_data
    WHERE
        "native-country" != 'United-States' -- double quotes for columns, single quotes for strs
    GROUP BY
        education,
        "education-num"
),

-- then table of us-born
us_born as (
    SELECT
        "education-num",
        education,
        COUNT(*) as count
    FROM
        adult_data
    WHERE
        "native-country" = 'United-States'
    GROUP BY
        education,
        "education-num"
),

-- then join of both on education level (with coalesce and full outer in case counts of 0)
joined as (
    SELECT
        nus."education-num",
        nus.education,
        COALESCE(nus.count,0) as non_us_count, --in case of zeros in a category
        COALESCE(us.count,0) as us_count
    FROM
        non_us_born nus
    FULL OUTER JOIN
        us_born us
    ON
        us.education = nus.education
)

-- then finally calculate percentages as well
SELECT
    "education-num",
    education,
    non_us_count,
    ROUND(SUM(non_us_count) * 100 / SUM(non_us_count) over (),2) as non_us_perc,
    us_count,
    ROUND(SUM(us_count) * 100 / SUM(us_count) over (),2) as us_perc
FROM
    joined
GROUP BY
    "education-num", education, non_us_count, us_count
ORDER BY
    "education-num" DESC;

Running query in 'postgresql:///adult_data'

16 rows affected.

education-num,education,non_us_count,non_us_perc,us_count,us_perc
16,Doctorate,85,2.51,328,1.12
15,Prof-school,74,2.18,502,1.72
14,Masters,196,5.78,1527,5.23
13,Bachelors,589,17.37,4766,16.34
12,Assoc-acdm,85,2.51,982,3.37
11,Assoc-voc,93,2.74,1289,4.42
10,Some-college,551,16.25,6740,23.11
9,HS-grad,799,23.56,9702,33.26
8,12th,68,2.01,365,1.25
7,11th,108,3.18,1067,3.66


Immediate observations are:

1. The Doctorate (education-num = 16) education level has twice the relative representation among non-U.S. born persons than those born in the U.S.

2. The Prof-school, Masters, and Bachelors (15 - 13) levels show more even representation.

3. Those U.S. born with Associates, some college, or complete high school (12 - 9) have greater representation.

4. 10th through 12th grade high school education (6 - 8) has somewhat even representation.

5. Non-U.S. born persons with an education level below 10th grade high school (<= 5) are much more populous than U.S. born.

**Q2 Answer: The extrema of the education levels (doctorate and those with 9th grade education or below) have a higher non-U.S. born relative population than U.S born. U.S. born persons with an Associate's degree, some college, or a high school diploma have greater representation. The remaining categories are largely even between both populations.**

### Q3: Does sex affect the likelihood of earning greater than \$50k in salary?

Recall that salary here is a binary variable (<=50k, or >50k) so we can't get a continuous distribution.

In [150]:
%%sql
SELECT
    sex,
    SUM(CASE WHEN salary LIKE '<=%' THEN 1 ELSE 0 END) as lte_50k,
    SUM(CASE WHEN salary LIKE '>%' THEN 1 ELSE 0 END) as gt_50k
FROM
    adult_data
GROUP BY
    sex;

Running query in 'postgresql:///adult_data'

2 rows affected.

sex,lte_50k,gt_50k
Female,9592,1179
Male,15128,6662


Let's re-express the above table with percentages while also showing the marginal distribution:

In [178]:
%%sql

-- get raw counts first
WITH salary_counts as (
    SELECT
        sex,
        SUM(CASE WHEN salary LIKE '<=%' THEN 1 ELSE 0 END) as lte_50k,
        SUM(CASE WHEN salary LIKE '>%' THEN 1 ELSE 0 END) as gt_50k
    FROM
        adult_data
    GROUP BY
        sex
),

total_counts as (
    SELECT
        COUNT(*)::NUMERIC as count
    FROM
        adult_data
),

-- reexpress counts as percentages
percs as (
    SELECT
        sc.sex,
        ROUND(lte_50k * 100 / tc.count,2) as lte_50k_perc,
        ROUND(gt_50k * 100/ tc.count,2) as gt_50k_perc,
        ROUND((lte_50k + gt_50k) / tc.count,2) as "col_sum"
    FROM
        salary_counts sc,
        total_counts tc
)

-- then shows percentages along with margins
SELECT
    percs.sex,
    lte_50k_perc,
    gt_50k_perc,
    lte_50k_perc + gt_50k_perc as "col_sum"
FROM
    percs

UNION ALL

SELECT
    'row_sum',
    SUM(lte_50k_perc),
    SUM(gt_50k_perc),
    SUM(lte_50k_perc) + SUM(gt_50k_perc)
FROM
    percs

Running query in 'postgresql:///adult_data'

3 rows affected.

sex,lte_50k_perc,gt_50k_perc,col_sum
Female,29.46,3.62,33.08
Male,46.46,20.46,66.92
row_sum,75.92,24.08,100.00


From the table, it's evident that women are 33% of the table data, 38% (= 29.46% / 75.92%) of those earning <=\\$50k in salary, and only 18% (= 3.62% / 20.46%) of those earning >\\$50k in salary. This means that in the >\\$50k salary category, men account for about 82\% of the data.

**Q3 Answer: Sex has a strong relationship between earning under or above $50k in salary.**

### Q4: For those earning a salary >\\$50k, is there a change in hours-per-week worked between different racial categories?

In [189]:
%%sql
SELECT
    ROUND(AVG("hours-per-week"),2) as avg_hours_per_week,
    ROUND(STDDEV_SAMP("hours-per-week"),2) as std_hours_per_week,
    COUNT(*) as count,
    race,
    salary
FROM
    adult_data
WHERE
    salary LIKE '>%'
GROUP BY
    race,
    salary

Running query in 'postgresql:///adult_data'

5 rows affected.

avg_hours_per_week,std_hours_per_week,count,race,salary
44.92,9.40,25,Other,>50K
45.56,11.03,7117,White,>50K
44.83,11.13,276,Asian-Pac-Islander,>50K
44.92,9.85,36,Amer-Indian-Eskimo,>50K
44.38,10.84,387,Black,>50K


The average number of hours worked per week is all almost within 1 hour of each racial group. The standard deviations are small for the Other and Amer-Indian-Eskimo categories, but these groups have very few counts relative to the other 3 

**Q4 Answer: There seems to be no relationship between average number of hours worked per week and different racial groups when earning >\\$50k in salary.**

# Hypothesis Testing

Each of the 4 questions were answered, but only with a cursory analysis. These answers are investigated and statistically validated in the R document adult_demographics_analysis_R.pdf.

# Clustering

A Kmodes clustering of this dataset is walked through in the AdultData_KModes.ipynb Jupyter notebook.

# Tableau

A tableau dashboard showing the trend of average hours worked per week by U.S. born individuals vs those not born in the U.S., the distribution of sex, workclass, and race, and the salary disparities by gender across racial categories can be found on the Tableau Public Server at https://tinyurl.com/bdej24j2.

*Screenshot of the original dashboard over the full dataset.*

![image](images/tableau_full.png)

*Screenshot showing the dashboard statistics for those not born in the United States.*

![image](images/tableau_nonusborn.png)

*Showing the visuals only corresponding to the Asian-Pacific-Islander racial category.*

![image](images/tableau_asianpacificislander.png)

*And the visuals for the US-born White, Black, and American-Indian-Eskimo racial categories.*

![image](images/tableau_whiteblackamericanindianeskimo_usborn.png)